## 추론(예측) 기반 기법과 신경망

## word2vec : 워드투벡터
2013년 구글의 토마스미콜로프(Tomas Mikolov)의 팀이 개발<br>
<b>word2vec</b> 알고리즘은 <b>신경망 모델</b>을 사용 하여 큰 텍스트 코퍼스에서 단어 연관성을 학습. 학습이 끝나면 이러한 모델은 동의어 단어를 감지하거나 부분 문장에 대한 추가 단어를 제안 할 수 있다. word2vec는 <b>벡터</b> 라고하는 특정 숫자 목록을 사용하여 각각의 고유 한 단어를 나타낸다 . 벡터는 간단한 수학적 함수 ( 벡터 간의 코사인 유사성 ) 가 해당 벡터가 나타내는 단어 간의 의미 유사성 수준을 나타내 도록 신중하게 선택 된다.



## [1] 신경망에서의 단어 처리

In [1]:
import numpy as np

text = 'You say goodbye and I say hello.'
# {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}
# 여기서 'you'만  one-hot 인코딩으로 표현

c = np.array([[1,0,0,0,0,0,0]])
print('c:\n',c)
W = np.random.randn(7,3)
print('W:\n',W)

h = np.matmul(c,W)  # (1,7) * (7,3) = (1,3)
print('h:\n',h)      

c:
 [[1 0 0 0 0 0 0]]
W:
 [[-0.57857855 -1.67459904  0.05558954]
 [ 0.29040516 -0.67341816 -0.81094513]
 [ 0.86535339  0.05519128 -0.0480388 ]
 [ 0.88541135  0.21701012  0.82416437]
 [ 0.295115    0.19340395  0.60828617]
 [ 1.02092804  0.64156864 -1.16876987]
 [-0.77889008  1.27165271  0.55451985]]
h:
 [[-0.57857855 -1.67459904  0.05558954]]


In [11]:
class MatMul:
    def __init__(self,W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None
        
    def forward(self,x):
        W, = self.params
        out = np.dot(x,W)
        self.x = x
        return out
        
    def backward(self,dout):
        W, =self.params
        dx = np.dot(dout,W.T)
        dW = np.dot(self.x.T, dout)
        self.grads[0] = dw
        return dx

In [12]:
text = 'You say goodbye and I say hello.'
# {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}
# 여기서 'you'만  one-hot 인코딩으로 표현

c = np.array([[1,0,0,0,0,0,0]])
print('c:\n',c)
W = np.random.randn(7,3)
print('W:\n',W)

layer = MatMul(W)
h = layer.forward(c)  # (1,7) * (7,3) = (1,3)
print('h:\n',h) 

c:
 [[1 0 0 0 0 0 0]]
W:
 [[-0.64447594  0.17357443  0.78685697]
 [ 0.27102534 -0.86601617  1.2378166 ]
 [ 2.79128591  0.70395061  0.37847108]
 [ 2.53486091 -0.43372363 -0.12090977]
 [ 0.62140344 -0.40362139 -0.17572194]
 [-0.60715039 -0.8835753   1.13418549]
 [-1.71422171  0.40905302  0.17731684]]
h:
 [[-0.64447594  0.17357443  0.78685697]]


## [2] 단순한 word2vec

### CBOW (Continuous Bag of Words) 모델

#### Word2Vec에는 CBOW(Continuous Bag of Words)와 Skip-Gram 두 가지 방식이 있다
- $ CBOW $ 는 주변에 있는 단어들을 가지고, 중간에 있는 단어들을 예측하는 방법 <br>
  타깃(target)은 중앙 단어 그 주변 단어들이 맥락(contexts)이다
- $ Skip-Gram $ 은 중간에 있는 단어로 주변 단어들을 예측하는 방법

#### BOW(Bag of Words) : 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터의 수치화 표현 방법

BOW를 만드는 과정<br>
(1) 우선, 각 단어의 고유한 인덱스(Index)를 부여한다.<br>
(2) 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터(Vector)를 만든다.<br>

"정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."<br>
('정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9) <br>
BOW: [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]  ==> '가' 와 '물가상승률' 은 2회 발생

https://wikidocs.net/22650

In [16]:
# (CBOW 전체구조의 Preview)
# 샘플 맥락 데이터 : 2개의 주변 단어를 맥락으로 중간 단어('say')를 예측

text = 'You say goodbye and I say hello.'

# 2개의 주변 단어를 one-hot 벡터 생성
c0 = np.array([[1,0,0,0,0,0,0]])  # 'you'   , (1,7)
c1 = np.array([[0,0,1,0,0,0,0]])  # 'goodbye' (1,7)


# 가중치 초기화
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

# 계층 생성
in_layer0 = MatMul(W_in)   
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

# 순전파
h0 = in_layer0.forward(c0)  # (1,7) * (7,3)  = (1,3)
h1 = in_layer0.forward(c1)  # (1,7) * (7,3)  = (1,3)
h = 0.5 * (h0 + h1)         # 입력층이 여러 개이면 전체를 평균
s = out_layer.forward(h)    # (1,3) * (3,7)  = (1,7) ,최종 출력
print(s,s.shape)            # (1, 7), 7개 단어의 스코어


[[-0.83104048  0.07745774  0.50899159  0.54389987  1.39547588  0.37293676
   0.34065541]] (1, 7)


## [3] 학습 데이터 준비